# Symbolic Solutions

Generates a table showing the symbolic models forms and solution rates

In [1]:
import pandas as pd
# import sympy 
from sympy import latex
from sympy.parsing.sympy_parser import parse_expr
df_results = pd.read_feather('results_sym_data_solns.feather')
df_results

,dataset,algorithm,random_state,process_time,training time (s),target_noise,feature_noise,true_model,model_size,symbolic_model,...,symbolic_error,symbolic_fraction,symbolic_error_is_zero,symbolic_error_is_constant,symbolic_fraction_is_constant,sympy_exception,training time (hr),r2_zero_test,data_group,symbolic_solution
0,feynman_III_10_19,AFP,14423,3936.777982,3948.102453,0.000,0.0,mom*sqrt(Bx**2 + By**2 + Bz**2),49,(((x_1+x_2)*(0.604*x_0))+((x_0*0.602)*x_3)),...,mom*(-0.604*Bx - 0.604*By - 0.602*Bz + sqrt(Bx...,(0.604*Bx + 0.604*By + 0.602*Bz)/sqrt(Bx**2 + ...,False,False,False,None,1.096695,0.991223,Feynman,False
1,feynman_III_10_19,AFP,14423,4372.954111,4378.418628,0.001,0.0,mom*sqrt(Bx**2 + By**2 + Bz**2),60,((x_0/(1.94/x_3))+((x_1*(((log(x_2)/x_2)-((log...,...,-Bx*(0.248 + (mom - mom/(By + 0.729)**3)/By + ...,(Bx*(0.248 + (mom - mom/(By + 0.729)**3)/By + ...,False,False,False,None,1.216227,0.990782,Feynman,False
2,feynman_III_10_19,AFP,14423,3165.802133,3177.535154,0.010,0.0,mom*sqrt(Bx**2 + By**2 + Bz**2),24,(x_0*(exp((x_1*0.28))+sqrt(|((x_3+((x_2+(-4.02...,...,mom*(sqrt(Bx**2 + By**2 + Bz**2) - 3.53*sqrt(0...,(3.53*sqrt(0.248*By + 0.274*Bz + 0.248*sin(By)...,False,False,False,None,0.882649,0.971934,Feynman,False
3,feynman_III_10_19,AFP,23654,3447.523341,3449.341744,0.001,0.0,mom*sqrt(Bx**2 + By**2 + Bz**2),55,((x_0*sqrt(|((x_3*sqrt(|((x_1+cos(log(((((x_3/...,...,-1.93*mom*sqrt(Bz*sqrt(-Bx - cos(log(-Bx*Bz - ...,(1.93*mom*sqrt(Bz*sqrt(-Bx - cos(log(-Bx*Bz - ...,False,False,False,None,0.958150,0.988344,Feynman,False
4,feynman_III_10_19,DSR,15795,824.784842,856.054502,0.001,0.0,mom*sqrt(Bx**2 + By**2 + Bz**2),12,x4 + (x1 + x2 + x3 + x4)*log(x1),...,-Bz + mom*sqrt(Bx**2 + By**2 + Bz**2) - (Bx + ...,(Bz + (Bx + By + Bz + mom)*log(mom))/(mom*sqrt...,False,False,False,None,0.237793,0.948120,Feynman,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43042,strogatz_vdp2,ITEA,14423,0.033981,427.956743,0.000,0.0,-x/10,10,-2.979387569990166e-16 + 2.8557885223268187e-1...,...,0,1.00,True,True,True,None,0.118877,1.000000,Strogatz,True
43043,strogatz_vdp2,SBP,860,18874.929026,18893.271010,0.001,0.0,-x/10,782,871640431564.907959+-0.152483*(((0.696291*(x0+...,...,None,None,False,False,False,integer division or modulo by zero,5.248131,0.999779,Strogatz,False
43044,strogatz_vdp2,SBP,860,23852.529831,23920.139926,0.010,0.0,-(1)/(10) * x,1000,-127.053383+-0.136279*(((((x0aq7.446000)-(-0.8...,...,None,None,False,False,False,None,6.644483,0.969929,Strogatz,False
43045,strogatz_vdp2,AIFeynman,15795,60.388924,26502.674970,0.010,0.0,-x/10,13,-0.096196192914*(x0-log(sin(exp(sin((pi)**(-1)...,...,0.00201 - 0.00403*x,-10*(-0.0959*x - 0.00201)/x,False,False,False,None,7.361854,0.998553,Strogatz,False


In [3]:
import pdb
from symbolic_utils import round_floats, complexity
from tqdm import tqdm

df_solns = df_results.loc[df_results.dataset.str.contains('strogatz')]
for (dataset,noise), dfg in tqdm(df_solns.groupby(['dataset','target_noise'])):
    tbl = []
#     print(80*'=')
    tbl.append('\\begin{tabularx}{\\textwidth}{l Z}') 
#     tbl.append('\t&\t'.join(['Name','Ground Truth'])) 
#     tbl.append('\\midrule') 
#     print(dataset,noise)
#     print(dfg['true_model'].dropna())
    tm = dfg['true_model'].unique()[0]
    model = round_floats(parse_expr(tm, evaluate=False))
    prefix = '\dot{x} = ' if '1' in dataset else '\dot{y} = '
    tbl.append('\t&\t'.join([dataset, '$'+prefix+latex(model)+'$']))
#     print('True Model:',latex(model))
    dfg_solns = dfg.loc[dfg.symbolic_solution==True]
    tbl.append('\\midrule') 
    tbl.append('Method&\tEstimated Model')
    soln_algs = []
    if len(dfg_solns)>0:
#         print('Solutions',60*'=')
#         tbl.append('\\multicolumn{2}{c}{Solutions}')
        for alg, dfga in dfg_solns.groupby('algorithm'):
            sm = dfga['simplified_symbolic_model'].values[0] 
            model = round_floats(parse_expr(sm, evaluate=False))
#             if str(model) == '0':
#                 pdb.set_trace()
#             print('  ',alg,'model:',model)
            mstr = '$'+prefix+latex(model)+'$'
            tbl.append('\t&\t'.join(['\\textbf{'+alg+'}', mstr]))
            soln_algs.append(alg)
        tbl.append('\\midrule') 
        
    df_notsolns = dfg.loc[dfg.symbolic_solution==False]
    df_notsolns = df_notsolns.loc[~df_notsolns.algorithm.isin(soln_algs)]
#     print('Non-solutions',56*'=')
#         tbl.append('Method ($R^2$)\t&\tNon-solutions')
    for alg, dfga in df_notsolns.groupby('algorithm'):
        guesses = dfga['simplified_symbolic_model'].values
        accs = dfga['r2_test'].values
        if len(guesses) == 0:
            continue
#             print(guesses) 
#             acc = dfga['r2_test'].values[0]
        guess_model = None
        best_acc = None
        for i,(g,acc) in enumerate(zip(guesses,accs)):
            if g:
                gm = round_floats(parse_expr(g, evaluate=False))
                if complexity(gm)>40 or 'NaN' in latex(gm):
                    continue
                else:
                    if best_acc==None or acc > best_acc:
                        print('adding gm w complexity',complexity(gm))
                        guess_model = latex(gm)
                        best_acc = acc


        if guess_model:
#             print('\t',guess_model[:80], '...' if len(guess_model)>80 else '')
            if str(guess_model)=='0': 
                continue
#                 guess_model = guess_model[:80]+ '\\dots' 
#                 for ob,cb in [('\\left(','\\right)'),('{','}'),('(',')')]:
#                     open_b = guess_model.count(ob)
#                     close_b = guess_model.count(cb)
#                     while open_b > close_b:
#                         guess_model += cb
#                         close_b += 1
#             tbl.append('\t&\t'.join([alg+' ({:1.2f})'.format(acc), '$'+guess_model+'$']))
            mstr = '$'+prefix+guess_model+'$'
            tbl.append('\t&\t'.join([alg, mstr]))

    #         for g in guess:
    #             print('\t',g[:80],'...' if len(g)>80 else '')
    #             print('\t',g)
    tbl.append('\\bottomrule') 
    tbl.append('\\end{tabularx}')
    tbl = [t+'\t\\\\' if all([s not in t for s in ['tabular','rule','hline']]) else t for t in tbl ]
#     tbl = ['\\begin{table}\n\\tiny\n\\centering\n']+tbl+['\\end{table}\n']
    filename = '../../paper/tables/results_sym_data/'+'_'.join([dataset,str(noise)])+'_sym_models.tex'
    with open(filename, 'w') as f:
        f.write('\n'.join(tbl).replace('_','\\_'))


  0%|          | 0/42 [00:00<?, ?it/s]

adding gm w complexity 26
adding gm w complexity 14
adding gm w complexity 38
adding gm w complexity 33
adding gm w complexity 33
adding gm w complexity 33
adding gm w complexity 25
adding gm w complexity 32
adding gm w complexity 14
adding gm w complexity 29
adding gm w complexity 35
adding gm w complexity 25
adding gm w complexity 25
adding gm w complexity 31
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 19
adding gm w complexity 30
adding gm w complexity 1
adding gm w complexity 34


  2%|▏         | 1/42 [00:06<04:36,  6.75s/it]

adding gm w complexity 20
adding gm w complexity 20
adding gm w complexity 33
adding gm w complexity 39
adding gm w complexity 38
adding gm w complexity 1
adding gm w complexity 5
adding gm w complexity 34
adding gm w complexity 14
adding gm w complexity 22
adding gm w complexity 37
adding gm w complexity 32
adding gm w complexity 32
adding gm w complexity 33
adding gm w complexity 29
adding gm w complexity 39
adding gm w complexity 35
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 25


  5%|▍         | 2/42 [00:14<04:44,  7.10s/it]

...

 95%|█████████▌| 40/42 [09:44<00:19,  9.56s/it]

adding gm w complexity 30
adding gm w complexity 34
adding gm w complexity 24
adding gm w complexity 5
adding gm w complexity 31


 98%|█████████▊| 41/42 [09:47<00:07,  7.46s/it]

adding gm w complexity 23
adding gm w complexity 9
adding gm w complexity 22
adding gm w complexity 28
adding gm w complexity 10
adding gm w complexity 12
adding gm w complexity 5
adding gm w complexity 27
adding gm w complexity 8
adding gm w complexity 6
adding gm w complexity 18
adding gm w complexity 5
adding gm w complexity 5
adding gm w complexity 36
adding gm w complexity 33


100%|██████████| 42/42 [10:08<00:00, 14.48s/it]


In [ ]:
df_solns = df_results.loc[df_results.dataset.str.contains('strogatz_bacres1')]
df_solns = df_solns.loc[df_solns.algorithm=='ITEA']
df_solns = df_solns.loc[df_solns.target_noise==0.0]
for (tm, sm), dfg in df_solns.groupby(['true_model','simplified_symbolic_model']):
    print(tm)
    print('\t',sm)

# strogatz

In [ ]:
df_strogatz = df_results.loc[df_results.dataset.str.contains('strogatz')]
tbl = ['\\begin{tabularx}{\\textwidth}{lr} \\\\'
       'Dataset 
      ]
for dataset, dfg in df_strogatz.groupby('dataset'):
    tbl.append(
        ''
    )
    

In [ ]:
df_sum = pd.read_csv('symbolic_dataset_results_sum.csv.gz') 
df_strogatz_sum = df_sum.loc[df_sum.dataset.str.contains('strogatz'),:] 
df_feynman_sum = df_sum.loc[df_sum.dataset.str.contains('feynman'),:] 
strogatz_solns = df_strogatz_sum.set_index(['dataset','target_noise','algorithm'])['symbolic_solution_rate_(%)'].unstack().round(0)
feynman_solns = df_feynman_sum.set_index(['dataset','target_noise','algorithm'])['symbolic_solution_rate_(%)'].unstack().round(0)
strogatz_solns.to_latex('../../paper/tables/strogatz_soln_pct.tex')

In [ ]:
df_sum.loc[df_sum.dataset.str.contains('strogatz'),:]